## Notes about the model:

- **Parametric supervised learning method** that models the probability of a binary outcome (Attrition) using a logistic function
- The algorithm predicts the likelihood of employee attrition by learning a linear combination of features, transformed through the sigmoid function to output probabilities between 0 and 1
- **Handles both numeric and categorical variables**, though scikit-learn implementation requires encoding categorical features as dummy variables (one-hot encoding), similar to other models
- Logistic Regression can suffer from **multicollinearity** when features are highly correlated, which can make coefficient interpretation unreliable and model performance unstable
- The model performs well when there is a **linear relationship between the log-odds of the outcome and the features**. Non-linear relationships may require feature engineering or polynomial terms
- **Class imbalance** is a significant challenge in this attrition problem, requiring techniques like SMOTE (Synthetic Minority Oversampling Technique) to improve recall for the minority class
- **Regularization parameters** (L1 or L2 penalties via the `C` parameter) help prevent overfitting by penalizing large coefficients, especially important when dealing with many features
- **Class weights** can be tuned to handle imbalanced datasets by assigning higher penalties to misclassifying the minority class
- Model provides **interpretable coefficients** that indicate the direction and strength of each feature's impact on attrition probability, making it valuable for business understanding

**Support:** [https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)


## Quick Overview: Logistic Regression for HR Attrition Prediction

This notebook implements a Logistic Regression model to predict employee attrition with a focus on handling class imbalance and optimizing probability thresholds for business use. It serves as a strong, interpretable baseline that performs very well in the global evaluation.

### Key Approach

- **Data Preprocessing**: Applies the shared preprocessing pipeline with imputation, scaling, encoding, and feature engineering (18 features total).
- **Class Imbalance Handling**: Uses SMOTE (Synthetic Minority Oversampling Technique) to increase the minority (leaver) representation in the training data from 16.2% to 33.3%.
- **Hyperparameter Tuning**: Runs GridSearchCV with 5‑fold StratifiedKFold across 50 parameter combinations to tune regularization strength, penalty, solver, and class weights.
- **Threshold Optimization**: Implements a dual threshold strategy on predicted probabilities:
  - **F1‑Optimized** (0.578): best balance between precision and recall.
  - **Recall‑Optimized** (0.400): prioritizes catching as many leavers as possible at the cost of more false positives (business‑focused setting).

### Results

**Best Model Configuration:**

- C: 0.1  
- Class Weight: `{0: 1, 1: 2}`  
- Penalty: L2  
- Solver: `lbfgs`  
- CV F1 Score: 0.7164  

**Test Set Performance (Logistic Regression):**

| Threshold              | Recall (Leavers) | Precision (Leavers) | Accuracy | Use Case                                      |
|------------------------|------------------|----------------------|----------|-----------------------------------------------|
| Default (0.50)         | 63.8%            | 39.5%                | 79%      | Baseline probability cutoff                   |
| F1‑Optimized (0.578)   | 53.2%            | 44.6%                | 82%      | Best overall F1 on test set                   |
| **Recall‑Optimized (0.400)** | **76.6%**    | **34.0%**            | 72%      | **Minimize missed departures (business focus)** |

At the recall‑optimized threshold, the model misses 11 leavers on the test set (out of 47), with 70 false alarms, offering a strong balance between catching leavers and controlling the number of flagged employees.

### Business Impact

The **Recall‑Optimized Logistic Regression model** is the top performer in the global evaluation at the business‑priority threshold, combining high recall with the best overall mix of precision, F1, and accuracy. It was selected as the primary deployment model, providing interpretable coefficients and clear probability scores that support proactive retention decisions and transparent HR communication.


In [ ]:
#!pip install imbalanced-learn


In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pathlib import Path

from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             precision_recall_curve, classification_report)
import joblib
import json
from imblearn.over_sampling import SMOTE

# Get the current notebook directory (notebooks folder)
base_path = Path.cwd()

# Get the project root (parent of notebooks folder)
project_root = base_path.parent

# Add the project root to Python path to find the scripts module
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import functions and feature list
from preprocessing import preprocess, selected_features

# SMOTE for oversampling
from imblearn.over_sampling import SMOTE


In [ ]:
def plot_roc(y_true, y_proba, title='ROC Curve'):
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(5,4))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0,1], [0,1],'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def plot_conf_matrix(y_true, y_pred, title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)
    plt.show()


In [ ]:
# --- Data Preprocessing ---
DATA_PATH = project_root / "data" / "raw" / "HR_Attrition_Dataset.csv"

# Call Preprocessing function to return split dataset
X_train, X_val, X_test, y_train, y_val, y_test, scaler, imputer = preprocess(data_path=DATA_PATH)


In [ ]:
# ============================================
# APPLY SMOTE + LOGISTIC REGRESSION HYPERPARAMETER TUNING
# ============================================

from imblearn.over_sampling import SMOTE

# Apply SMOTE to training data before GridSearchCV
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("=" * 60)
print("SMOTE OVERSAMPLING RESULTS")
print("=" * 60)
print(f"Training set before SMOTE: {X_train.shape[0]} samples")
print(f"Training set after SMOTE: {X_train_smote.shape[0]} samples")
print(f"\nClass distribution before SMOTE:")
print(y_train.value_counts(normalize=True).round(3))
print(f"\nClass distribution after SMOTE:")
print(pd.Series(y_train_smote).value_counts(normalize=True).round(3))
print("=" * 60)

# Set up StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Expanded hyperparameter grid with class_weight tuning
# Note: With SMOTE, class_weight becomes less critical but still worth tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear'],
    'class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

# Base classifier
clf = LogisticRegression(max_iter=1000, random_state=42)

# GridSearchCV with F1 scoring (for hyperparameters more balanced)
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='f1',
    cv=skf,
    verbose=2,
    n_jobs=-1
)

# Fit model with SMOTE-resampled data
print("\n" + "=" * 60)
print("RUNNING GRIDSEARCHCV WITH SMOTE DATA")
print("=" * 60)
grid_search.fit(X_train_smote, y_train_smote)

print("\n" + "=" * 60)
print("GRIDSEARCHCV RESULTS")
print("=" * 60)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score (CV): {grid_search.best_score_:.4f}")
print("=" * 60)

# Best estimator
best_logreg = grid_search.best_estimator_


In [ ]:
# --- Get Cross-Validation Results ---
cv_results = pd.DataFrame(grid_search.cv_results_)
leaderboard = cv_results.loc[:, ['params', 'mean_test_score', 'std_test_score']]
leaderboard = leaderboard.rename(columns={'mean_test_score': 'mean_F1', 'std_test_score': 'std_F1'})
leaderboard = leaderboard.sort_values(by="mean_F1", ascending=False)
print("\nTop 10 Hyperparameter Combinations:")
print(leaderboard.head(10))

In [ ]:
# ============================================================
# THRESHOLD OPTIMIZATION - DUAL STRATEGY
# 1. F1-Optimized: Best balance between precision and recall
# 2. Recall-Optimized: Minimize surprise departures
# ============================================================

from sklearn.metrics import precision_recall_curve

# Get predicted probabilities on validation set
y_val_proba = best_logreg.predict_proba(X_val)[:, 1]

# Calculate precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_val, y_val_proba)

# ---- F1-OPTIMIZED THRESHOLD ----
# Calculate F1 score at each threshold
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1] + 1e-10)
best_f1_idx = np.argmax(f1_scores)
f1_optimal_threshold = thresholds[best_f1_idx]
f1_achieved_recall = recalls[best_f1_idx]
f1_achieved_precision = precisions[best_f1_idx]
f1_achieved_f1 = f1_scores[best_f1_idx]

# ---- RECALL-OPTIMIZED THRESHOLD ----
TARGET_RECALL = 0.70  

valid_indices = np.where(recalls[:-1] >= TARGET_RECALL)[0]

if len(valid_indices) > 0:
    best_idx = valid_indices[np.argmax(precisions[:-1][valid_indices])]
    recall_optimal_threshold = thresholds[best_idx]
    recall_achieved_recall = recalls[best_idx]
    recall_achieved_precision = precisions[best_idx]
else:
    recall_optimal_threshold = thresholds[0]
    recall_achieved_recall = recalls[0]
    recall_achieved_precision = precisions[0]
    print(f"WARNING: Target recall {TARGET_RECALL:.0%} not achievable. Using lowest threshold.")

print("=" * 60)
print("THRESHOLD OPTIMIZATION RESULTS")
print("=" * 60)
print(f"\n{'F1-OPTIMIZED THRESHOLD':^60}")
print(f"Threshold: {f1_optimal_threshold:.3f}")
print(f"Precision: {f1_achieved_precision:.2%}")
print(f"Recall: {f1_achieved_recall:.2%}")
print(f"F1 Score: {f1_achieved_f1:.3f}")

print(f"\n{'RECALL-OPTIMIZED THRESHOLD':^60}")
print(f"Target Recall: {TARGET_RECALL:.0%}")
print(f"Threshold: {recall_optimal_threshold:.3f}")
print(f"Precision: {recall_achieved_precision:.2%}")
print(f"Recall: {recall_achieved_recall:.2%}")
print("=" * 60)

# After testing we decided to override with more aggressive threshold (the difference wouldn't be agressive enough)
recall_optimal_threshold = 0.40

print(f"OVERRIDDEN Recall-Optimized Threshold: {recall_optimal_threshold:.3f}")
print(f"(Manual override to account for validation-test gap)")

In [ ]:
# ============================================================
# VISUALIZATION: Precision-Recall Trade-off by Threshold
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Precision and Recall vs Threshold
ax1 = axes[0]
ax1.plot(thresholds, precisions[:-1], 'b-', label='Precision', linewidth=2)
ax1.plot(thresholds, recalls[:-1], 'g-', label='Recall', linewidth=2)
ax1.plot(thresholds, f1_scores, 'm-', label='F1 Score', linewidth=2, alpha=0.7)
ax1.axhline(y=TARGET_RECALL, color='r', linestyle='--', alpha=0.5, 
            label=f'Target Recall ({TARGET_RECALL:.0%})')
ax1.axvline(x=0.5, color='gray', linestyle=':', alpha=0.5, label='Default (0.5)')
ax1.axvline(x=f1_optimal_threshold, color='purple', linestyle='--', alpha=0.7, 
            label=f'F1-Optimized ({f1_optimal_threshold:.3f})')
ax1.axvline(x=recall_optimal_threshold, color='orange', linestyle='--', alpha=0.7, 
            label=f'Recall-Optimized ({recall_optimal_threshold:.3f})')
ax1.set_xlabel('Threshold', fontsize=12)
ax1.set_ylabel('Score', fontsize=12)
ax1.set_title('Precision, Recall & F1 at Different Thresholds', fontsize=14)
ax1.legend(loc='center right', fontsize=9)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1.05])
ax1.grid(True, alpha=0.3)

# Plot 2: Business Impact - Leavers caught at each threshold
ax2 = axes[1]
n_actual_leavers = y_val.sum()
leavers_caught = recalls[:-1] * n_actual_leavers

ax2.plot(thresholds, leavers_caught, 'g-', linewidth=2)
ax2.axvline(x=0.5, color='gray', linestyle=':', alpha=0.5, label='Default (0.5)')
ax2.axvline(x=f1_optimal_threshold, color='purple', linestyle='--', alpha=0.7,
            label=f'F1-Optimized')
ax2.axvline(x=recall_optimal_threshold, color='orange', linestyle='--', alpha=0.7,
            label=f'Recall-Optimized')
ax2.set_xlabel('Threshold', fontsize=12)
ax2.set_ylabel('Number of Employees', fontsize=12)
ax2.set_title(f'Leavers Caught at Different Thresholds\n(Total actual leavers: {n_actual_leavers:.0f})', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# VALIDATION SET EVALUATION - ALL THREE THRESHOLDS
# ============================================================

# Predictions with all three thresholds
y_val_pred_default = (y_val_proba >= 0.5).astype(int)
y_val_pred_f1 = (y_val_proba >= f1_optimal_threshold).astype(int)
y_val_pred_recall = (y_val_proba >= recall_optimal_threshold).astype(int)

print("=" * 60)
print("VALIDATION SET - DEFAULT THRESHOLD (0.5)")
print("=" * 60)
print(classification_report(y_val, y_val_pred_default, target_names=['Stay', 'Leave']))

print("=" * 60)
print(f"VALIDATION SET - F1-OPTIMIZED THRESHOLD ({f1_optimal_threshold:.3f})")
print("=" * 60)
print(classification_report(y_val, y_val_pred_f1, target_names=['Stay', 'Leave']))

print("=" * 60)
print(f"VALIDATION SET - RECALL-OPTIMIZED THRESHOLD ({recall_optimal_threshold:.3f})")
print("=" * 60)
print(classification_report(y_val, y_val_pred_recall, target_names=['Stay', 'Leave']))


In [ ]:
# ============================================================
# TEST SET EVALUATION - ALL THREE THRESHOLDS
# ============================================================

# Get test predictions
y_test_proba = best_logreg.predict_proba(X_test)[:, 1]
y_test_pred_default = (y_test_proba >= 0.5).astype(int)
y_test_pred_f1 = (y_test_proba >= f1_optimal_threshold).astype(int)
y_test_pred_recall = (y_test_proba >= recall_optimal_threshold).astype(int)

print("=" * 60)
print("TEST SET - DEFAULT THRESHOLD (0.5)")
print("=" * 60)
print(classification_report(y_test, y_test_pred_default, target_names=['Stay', 'Leave']))

print("=" * 60)
print(f"TEST SET - F1-OPTIMIZED THRESHOLD ({f1_optimal_threshold:.3f})")
print("=" * 60)
print(classification_report(y_test, y_test_pred_f1, target_names=['Stay', 'Leave']))

print("=" * 60)
print(f"TEST SET - RECALL-OPTIMIZED THRESHOLD ({recall_optimal_threshold:.3f})")
print("=" * 60)
print(classification_report(y_test, y_test_pred_recall, target_names=['Stay', 'Leave']))

# Business metrics comparison
cm_default = confusion_matrix(y_test, y_test_pred_default)
cm_f1 = confusion_matrix(y_test, y_test_pred_f1)
cm_recall = confusion_matrix(y_test, y_test_pred_recall)

print("\n" + "=" * 75)
print("BUSINESS IMPACT COMPARISON - TEST SET")
print("=" * 75)
print(f"{'Metric':<30} {'Default':>12} {'F1-Opt':>12} {'Recall-Opt':>12}")
print("-" * 75)
print(f"{'Threshold':<30} {'0.500':>12} {f1_optimal_threshold:>12.3f} {recall_optimal_threshold:>12.3f}")
print(f"{'Leavers Caught (TP)':<30} {cm_default[1,1]:>12} {cm_f1[1,1]:>12} {cm_recall[1,1]:>12}")
print(f"{'Missed Departures (FN)':<30} {cm_default[1,0]:>12} {cm_f1[1,0]:>12} {cm_recall[1,0]:>12}")
print(f"{'False Alarms (FP)':<30} {cm_default[0,1]:>12} {cm_f1[0,1]:>12} {cm_recall[0,1]:>12}")
print(f"{'Correct Stay Predictions (TN)':<30} {cm_default[0,0]:>12} {cm_f1[0,0]:>12} {cm_recall[0,0]:>12}")
print("=" * 75)

# Calculate percentages from the confusion matrices and create visual table
total_leavers = cm_default[1,0] + cm_default[1,1]
print(f"\n{'Recall (% Leavers Caught)':<30} {cm_default[1,1]/total_leavers:>11.1%} {cm_f1[1,1]/total_leavers:>12.1%} {cm_recall[1,1]/total_leavers:>12.1%}")
print(f"{'Precision (% Correct Flags)':<30} {cm_default[1,1]/(cm_default[1,1]+cm_default[0,1]):>11.1%} {cm_f1[1,1]/(cm_f1[1,1]+cm_f1[0,1]):>12.1%} {cm_recall[1,1]/(cm_recall[1,1]+cm_recall[0,1]):>12.1%}")


In [ ]:
# ============================================
# VISUALIZE CONFUSION MATRICES WITH PERCENTAGES + ROC CURVE
# ============================================

fig = plt.figure(figsize=(20, 5))
gs = fig.add_gridspec(1, 4, width_ratios=[1, 1, 1, 1.2])

# First 3 subplots for confusion matrices
axes = [fig.add_subplot(gs[0, i]) for i in range(3)]

# Calculate confusion matrices
cm_default = confusion_matrix(y_test, y_test_pred_default)
cm_f1 = confusion_matrix(y_test, y_test_pred_f1)
cm_recall = confusion_matrix(y_test, y_test_pred_recall)

thresholds_data = [
    ("Default (0.50)", y_test_pred_default, cm_default, 'Blues'),
    (f"F1-Opt ({f1_optimal_threshold:.2f})", y_test_pred_f1, cm_f1, 'Purples'),
    (f"Recall-Opt ({recall_optimal_threshold:.2f})", y_test_pred_recall, cm_recall, 'Greens')
]

for idx, (title, y_pred, cm, cmap) in enumerate(thresholds_data):
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create annotations with counts and percentages
    annot = np.array([[f'{cm[i,j]}\n({cm_percent[i,j]:.1f}%)' 
                       for j in range(cm.shape[1])] 
                      for i in range(cm.shape[0])])
    
    sns.heatmap(cm, annot=annot, fmt='', cmap=cmap,
                xticklabels=['Stay', 'Leave'],
                yticklabels=['Stay', 'Leave'],
                cbar=False, ax=axes[idx],
                annot_kws={'size': 12, 'weight': 'bold'})
    
    axes[idx].set_title(title, fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Predicted', fontsize=10)
    axes[idx].set_ylabel('Actual', fontsize=10)
    
    # Add recall at bottom
    recall = recall_score(y_test, y_pred)
    axes[idx].text(0.5, -0.15, f'Recall: {recall:.1%}', 
                   transform=axes[idx].transAxes,
                   fontsize=11, fontweight='bold',
                   horizontalalignment='center')

# Fourth subplot for ROC curve
ax_roc = fig.add_subplot(gs[0, 3])

# Calculate ROC curve
fpr, tpr, roc_thresholds = roc_curve(y_test, y_test_proba)
auc_score = roc_auc_score(y_test, y_test_proba)
print(auc_score)

# Plot ROC curve
ax_roc.plot(fpr, tpr, color='darkblue', lw=2, label=f'ROC curve (AUC = {auc_score:.3f})')
ax_roc.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')

# Mark the threshold points
for threshold_name, y_pred, color, marker in [
    ("Default", y_test_pred_default, 'red', 'o'),
    ("F1-Opt", y_test_pred_f1, 'green', 's'),
    ("Recall-Opt", y_test_pred_recall, 'purple', '^')
]:
    cm = confusion_matrix(y_test, y_pred)
    fpr_point = cm[0, 1] / (cm[0, 0] + cm[0, 1])  # FP / (TN + FP)
    tpr_point = cm[1, 1] / (cm[1, 0] + cm[1, 1])  # TP / (FN + TP)
    ax_roc.scatter([fpr_point], [tpr_point], s=100, c=color, marker=marker, 
                   edgecolors='black', linewidths=1.5, label=threshold_name, zorder=5)

ax_roc.set_xlim([0.0, 1.0])
ax_roc.set_ylim([0.0, 1.05])
ax_roc.set_xlabel('False Positive Rate', fontsize=11, fontweight='bold')
ax_roc.set_ylabel('True Positive Rate', fontsize=11, fontweight='bold')
ax_roc.set_title('ROC Curve', fontsize=12, fontweight='bold')
ax_roc.legend(loc='lower right', fontsize=9)
ax_roc.grid(alpha=0.3)

plt.suptitle('Logistic Regression - Confusion Matrices & ROC Curve (Test Set)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Get feature importance from coefficients
feature_names = X_train.columns
coefficients = best_logreg.coef_[0]
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

print("\n--- Top 10 Most Important Features ---")
print(feature_importance.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(10)
colors = ['green' if c > 0 else 'red' for c in top_features['coefficient']]
plt.barh(top_features['feature'][::-1], top_features['coefficient'][::-1], color=colors[::-1])
plt.xlabel('Coefficient')
plt.title('Top 10 Feature Importance (Logistic Regression Coefficients)')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# SAVE MODEL AND METADATA
# ============================================

# Create models directory at project root
MODEL_PATH = project_root / 'models'
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# Save the trained model
model_filename = MODEL_PATH / 'logistic_regression_best.pkl'
joblib.dump(best_logreg, model_filename)

# Save scaler and imputer (needed for deployment)
joblib.dump(scaler, MODEL_PATH / 'scaler.pkl')
joblib.dump(imputer, MODEL_PATH / 'imputer.pkl')
print(f"✓ Model saved to {model_filename}")

# Save thresholds and metadata
model_metadata = {
    'model_type': 'Logistic Regression',
    'best_params': grid_search.best_params_,
    'cv_f1_score': float(grid_search.best_score_),
    'test_auc_score': float(auc_score),
    'thresholds': {
        'default': 0.5,
        'f1_optimized': float(f1_optimal_threshold),
        'recall_optimized': float(recall_optimal_threshold)
    },
    'target_recall': float(TARGET_RECALL),
    'feature_names': list(X_train.columns)
}

metadata_filename = MODEL_PATH / 'logreg_model_metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(model_metadata, f, indent=4)
print(f"✓ Metadata saved to {metadata_filename}")

print(f"\n{'='*60}")
print("MODEL ARTIFACTS SAVED")
print(f"{'='*60}")
print(f"Model file: {model_filename}")
print(f"Metadata file: {metadata_filename}")
print(f"{'='*60}")
